In [1]:
%cd ../airflow
%pwd

/home/welberthime/Development/fiap/cartolafc/airflow


'/home/welberthime/Development/fiap/cartolafc/airflow'

In [2]:
from include import RawExtractor

extractor = RawExtractor(
    base_url="https://api.github.com/repos/henriquepgomide/caRtola/contents/data",
    path="./data/raw",
)

extractor.extract_static_files("2016")
extractor.extract_dynamic_files("2016")

In [4]:
import pandas as pd

In [5]:
clubes_df = pd.read_csv("./data/raw/2016/times_ids/1.csv")
clubes_df = clubes_df[["id", "cod.2018"]].rename(
    columns={"cod.2018": "olderID", "id": "clubeID"}
).drop_duplicates()

partidas_df = pd.read_csv("./data/raw/2016/partidas_ids/1.csv")

partidas_df = partidas_df.merge(
    right=clubes_df,
    left_on="CasaID",
    right_on="olderID",
    suffixes=(None, "_mandante"),
)

partidas_df = partidas_df.merge(
    right=clubes_df,
    left_on="VisitanteID",
    right_on="olderID",
    suffixes=(None, "_visitante"),
)

partidas_df["temporada"] = 2016
partidas_df = partidas_df.rename(
    columns={
        "ID": "partidaID",
        "clubeID": "clubeMandanteID",
        "clubeID_visitante": "clubeVisitanteID",
        "PlacarCasa": "mandantePlacar",
        "PlacarVisitante": "visitantePlacar",
        "Rodada": "rodada",
        "Resultado": "resultado",
    }
)

partidas_df = partidas_df[[
    "partidaID",
    "clubeMandanteID",
    "clubeVisitanteID",
    "mandantePlacar",
    "visitantePlacar",
    "rodada",
    "resultado",
    "temporada",
]]

partidas_df

,partidaID,clubeMandanteID,clubeVisitanteID,mandantePlacar,visitantePlacar,rodada,resultado,temporada
0,201233,327,266,0.0,1.0,1,Visitante,2016
1,201296,344,266,0.0,1.0,21,Visitante,2016
2,201449,277,266,2.0,1.0,29,Casa,2016
3,201497,293,266,1.0,0.0,16,Casa,2016
4,201560,283,266,4.0,2.0,34,Casa,2016
...,...,...,...,...,...,...,...,...
375,201491,264,327,2.0,0.0,31,Casa,2016
376,201375,275,327,2.0,0.0,10,Casa,2016
377,201320,285,327,3.0,1.0,7,Casa,2016
378,201438,294,327,3.0,0.0,28,Casa,2016


In [12]:
clubes_df = pd.read_csv("./data/raw/2016/times_ids/1.csv")
clubes_df = clubes_df[["id", "cod.2018"]].rename(
    columns={"cod.2018": "olderID", "id": "clubeID"}
).drop_duplicates()

scouts_df = pd.read_csv("./data/raw/2016/scouts_raw/1.csv")

scouts_df = scouts_df.merge(
    right=clubes_df,
    left_on="ClubeID",
    right_on="olderID",
)

scouts_df = scouts_df.rename(
    columns={
        'AtletaID': "atletaID",
        'Rodada': "rodada",
        'Pontos': "pontos",
        'PontosMedia': "pontosMedia",
        'Preco': "preco", 
        'PrecoVariacao': "precoVariacao",
    },
)

house_by_rounds = partidas_df[["partidaID", "rodada", "clubeMandanteID"]].rename(
    columns={"clubeMandanteID": "clubeID"}
)
visitor_by_rounds = partidas_df[["partidaID", "rodada", "clubeVisitanteID"]].rename(
    columns={"clubeVisitanteID": "clubeID"}
)
club_by_rounds = pd.concat([house_by_rounds, visitor_by_rounds])

scouts_df = scouts_df.merge(
    right=club_by_rounds,
    on=["rodada", "clubeID"],
)

scouts_df = scouts_df[[
    "clubeID",
    "partidaID",
    "atletaID",
    "pontos",
    "pontosMedia",
    "preco",
    "precoVariacao",
    "FS",
    "PE",
    "A",
    "FT",
    "FD",
    "FF",
    "G",
    "I",
    "PP",
    "RB",
    "FC",
    "GC",
    "CA",
    "CV",
    "SG",
    "DD",
    "DP",
    "GS",
]]

scouts_df

,clubeID,partidaID,atletaID,pontos,pontosMedia,preco,precoVariacao,FS,PE,A,...,PP,RB,FC,GC,CA,CV,SG,DD,DP,GS
0,294,201231,37623,5.3,5.30,6.84,2.84,0,3,0,...,0,1,1,0,0,0,1,0,0,0
1,294,201231,92192,0.0,0.00,2.00,0.00,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,294,201231,78464,0.0,0.00,2.00,0.00,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,294,201231,74210,0.1,0.10,3.01,-0.99,3,3,0,...,0,0,1,0,0,0,0,0,0,0
4,294,201231,70929,0.0,0.00,13.00,0.00,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30416,293,201604,71479,0.2,1.77,8.23,0.15,0,0,0,...,0,0,1,0,0,0,0,0,0,0
30417,293,201604,96788,0.0,0.00,1.00,0.00,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30418,293,201604,84190,0.0,0.35,1.70,0.00,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30419,293,201604,91204,0.0,0.00,1.00,0.00,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
clubes_df = pd.read_csv("./data/raw/2016/times_ids/1.csv")
clubes_df = clubes_df[["id", "cod.2018"]].rename(
    columns={"cod.2018": "olderID", "id": "clubeID"}
).drop_duplicates()

atletas_df = pd.read_csv("./data/raw/2016/jogadores/1.csv")

atletas_df = atletas_df.merge(
    right=clubes_df,
    left_on="ClubeID",
    right_on="olderID",
)

atletas_df.drop_duplicates(inplace=True)
atletas_df.rename(
    columns={
        "ID": "atletaID",
        "Apelido": "apelido",
        "PosicaoID": "posicaoID",
    },
    inplace=True
)

atletas_df = atletas_df[[
    "atletaID",
    "clubeID",
    "posicaoID",
    "apelido",
]]

atletas_df

,atletaID,clubeID,posicaoID,apelido
0,37623,294,3,Rafael Marques
1,37792,294,4,Edinho
2,95855,294,4,Kady
3,68807,294,5,Neto Berola
4,68697,294,4,Bernardo
...,...,...,...,...
996,71479,293,4,Lucho González
997,96788,293,4,Victor Feitosa
998,84190,293,5,Guilherme Schettine
999,91204,293,1,Warleson


In [9]:
posicoes_df = pd.read_csv("./data/raw/2016/posicoes_ids/1.csv")
posicoes_df.rename(
    columns={
        "Cod": "posicaoID",
        "Position": "nome",
        "abbr": "abreviacao",
    },
    inplace=True,
)

posicoes_df = posicoes_df[[
    "posicaoID",
    "nome",
    "abreviacao",
]]

posicoes_df

,posicaoID,nome,abreviacao
0,1,Goleiro,gol
1,2,Lateral,lat
2,3,Zagueiro,zag
3,4,Meia,mei
4,5,Atacante,ata
5,6,Técnico,tec


In [8]:
clubes_df = pd.read_csv("./data/raw/2016/times_ids/1.csv")
clubes_df.rename(
    columns={
        "id": "clubeID",
        "nome.cbf": "nome",
    },
    inplace=True,
)

clubes_df = clubes_df[["clubeID", "nome", "abreviacao"]]
clubes_df.drop_duplicates(subset=["clubeID"], keep="last", inplace=True)

clubes_df

,clubeID,nome,abreviacao
0,327,América - MG,AME
1,200,America - RN,OUT
2,373,Atlético - GO,ATL
3,282,Atlético - MG,ATL
6,293,Athlético-PR,ATL
7,314,Avaí - SC,AVA
8,265,Bahia - BA,BAH
9,263,Botafogo - RJ,BOT
10,203,Brasiliense,BRA
13,354,Ceará - CE,CEA
